In [1]:
!pip install refinitiv.data

In [2]:
import pandas as pd
import numpy as np
import os
import refinitiv.data as rd
from refinitiv.data.content import historical_pricing
import logging.config
import datetime
import json

In [3]:
APP_KEY                     = 'aace7028dd2744ab85f73cfceefa2350592ac03b'
RDP_LOGIN                   = '*********************'
RDP_PASSWORD                = "***********"

# praticamente la chiave da sola non basta. O installi anche Eikon Desktop e fai accesso lì, allora puoi usare solo la APIKEY, altrimenti metti la password (e speriamo che basti!)

# REALTIME_DISTRIBUTION_SERVICE_HOST      = 'THE_HOST:PORT_OF_YOUR_RTDS_HOST'  
# REALTIME_DISTRIBUTION_SERVICE_USER_NAME = 'YOUR_USER_NAME_ON_YOUR_RTDS_HOST' 

## Open the default session

To open the default session ensure you have 
* a '*refinitiv-data.config.json*' in the ***Configuration*** directory 
* **populated the file with your credentials**
* and specified a 'default' session in the config file    

**Note**: To request Historical Pricing data you must be using an RDP Platform (Cloud) session or a Desktop Session(Eikon/Workspace).   
It is not currently possible to request historical data from a deployed Session

Questa pappardella è solo per dirmi che in pratica bisogna installare il programma, fuck!


### Historical Price Events

NOTE: We have not specified any start or end times, so the request defaults to the 20 most recent events

In [41]:
# rd.open_session()
rd.open_session(app_key = APP_KEY)

<refinitiv.data.session.Definition object at 0x2245cf03e80 {name='workspace'}>

In [27]:
etf = pd.read_excel("elenco ETF.xlsx")
etf

,Name,Borsa,Presente,ISIN,RIC,Lipper Classification Scheme,Instrument Type,Domicile,Currency,Exchange iNAV
0,Alerian Midstream Energy Dividend UCITS ETF Dis,Milano,True,IE00BKPTXQ89,MMLP.MI,Equity Theme - Infrastructure,Equity ETF,Ireland,Euro,NaN
1,AM MSCI EUR IG SRI SusExFossil CrpBd UCITS ETF,Milano,True,IE000NPK2VI6,FAMSFFA.MI,Bond EUR Corporates,Exchange-Traded Fund,Ireland,Euro,NaN
2,AM MSCI EUR IG SRI SusExFossil1-3yCrpBdUCITSETF,Milano,True,IE000W6ZFH20,FAMSF3A.MI,Bond EUR Corporates Short Term,Exchange-Traded Fund,Ireland,Euro,NaN
3,Amndi Blmbrg Eqlwght ComdtexAgrcl UCITSETF DHE...,Milano,True,LU1900069219,COMH.MI,Commodity Blended,Commodity ETF,Luxembourg,Euro,NaN
4,Amundi Blbrg Equlwght Commdt ex-Agricl UCITS E...,Milano,True,LU1829218749,COMO.MI,Commodity Blended,Commodity ETF,Luxembourg,Euro,INCRBINAV.PA^F23
...,...,...,...,...,...,...,...,...,...,...
1820,Xtrackers Wld Net Zero Pway Par Algnd UCITS ET...,Milano,True,IE000UZCJS58,XNZW.MI,Equity Global Sm&Mid Cap,Equity ETF,Ireland,Euro,NaN
1821,ETC Group Physical Bitcoin,XETRA,True,NaN,BTCE.DE,NaN,ETN,NaN,Euro,NaN
1822,21Shares Bitcoin ETP,XETRA,True,NaN,2BTC.DE,NaN,ETN,NaN,Euro,NaN
1823,21Shares Ethereum Staking ETP,XETRA,True,NaN,ETHA.DE,NaN,ETN,NaN,Euro,NaN


In [26]:
response = historical_pricing.summaries.Definition(
#   "ZETH.DE",
   ["BTCE.DE", "2BTC.DE", "ETHA.DE", "ZETH.DE"],
#   ["VWCE.MI", "EQQQ.MI"],
    count=30000,
    interval=historical_pricing.Intervals.DAILY,
    # fields = ["OPEN_PRC","BID","ASK","HIGH_1", "LOW_1", "TRDPRC_1","TRNOVR_UNS"]
#    fields = "TRDPRC_1",
     fields = ["OPEN_PRC","HIGH_1", "LOW_1", "TRDPRC_1","TRNOVR_UNS"]
    ).get_data()
# Extract in DataFrame format
response.data.df

BTCE.DE                                        2BTC.DE          \
           OPEN_PRC  HIGH_1   LOW_1 TRDPRC_1   TRNOVR_UNS OPEN_PRC  HIGH_1   
Date                                                                         
2020-06-18   8.5315  8.5545    8.42     <NA>  267458.3545     <NA>    <NA>   
2020-06-19    8.333  8.4185   8.333     <NA>     85868.75     <NA>    <NA>   
2020-06-22    8.451  8.4955  8.4395     <NA>    150195.77     <NA>    <NA>   
2020-06-23    8.558   8.592   8.541     <NA>    46868.787     <NA>    <NA>   
2020-06-24   8.5795  8.5795   8.265     <NA>   116641.181     <NA>    <NA>   
...             ...     ...     ...      ...          ...      ...     ...   
2023-12-21    37.24  37.512   36.83     <NA>  3653076.374   13.768   13.82   
2023-12-22   36.968    37.1  36.614     <NA>  4441907.434   13.605  13.672   
2023-12-27   35.886  36.458   35.68     <NA>  9916520.314   13.243  13.445   
2023-12-28    36.15   36.18  35.536     <NA>  3796368.832   13.344  13.344   
2023-12-29   35.714   36.14  35.584     <NA>  1286235.178   13.138    13.3   

                                          ETHA.DE                           \
             LOW_1 TRDPRC_1   TRNOVR_UNS OPEN_PRC  HIGH_1   LOW_1 TRDPRC_1   
Date                                                                         
2020-06-18    <NA>     <NA>         <NA>     <NA>    <NA>    <NA>     <NA>   
2020-06-19    <NA>     <NA>         <NA>     <NA>    <NA>    <NA>     <NA>   
2020-06-22    <NA>     <NA>         <NA>     <NA>    <NA>    <NA>     <NA>   
2020-06-23    <NA>     <NA>         <NA>     <NA>    <NA>    <NA>     <NA>   
2020-06-24    <NA>     <NA>         <NA>     <NA>    <NA>    <NA>     <NA>   
...            ...      ...          ...      ...     ...     ...      ...   
2023-12-21  13.567     <NA>  1202254.614   22.322  22.908    22.3     <NA>   
2023-12-22  13.532     <NA>   1365890.99   22.914   23.49  22.914     <NA>   
2023-12-27  13.136     <NA>  1351115.445   22.362   23.57    22.3     <NA>   
2023-12-28  13.092     <NA>  1226857.704   23.712    24.0  23.462     <NA>   
2023-12-29  13.101     <NA>   529439.592   23.342  23.744  23.342     <NA>   

                        ZETH.DE                                       
            TRNOVR_UNS OPEN_PRC  HIGH_1   LOW_1 TRDPRC_1  TRNOVR_UNS  
Date                                                                  
2020-06-18        <NA>     <NA>    <NA>    <NA>     <NA>        <NA>  
2020-06-19        <NA>     <NA>    <NA>    <NA>     <NA>        <NA>  
2020-06-22        <NA>     <NA>    <NA>    <NA>     <NA>        <NA>  
2020-06-23        <NA>     <NA>    <NA>    <NA>     <NA>        <NA>  
2020-06-24        <NA>     <NA>    <NA>    <NA>     <NA>        <NA>  
...                ...      ...     ...     ...      ...         ...  
2023-12-21  451131.802   19.419  19.879  19.343     <NA>   123490.09  
2023-12-22   631979.74   19.942  20.278  19.926     <NA>  338829.136  
2023-12-27  486282.824   19.416  20.506  19.355     <NA>  747180.025  
2023-12-28  628054.026     20.5  20.744  20.388     <NA>  239862.094  
2023-12-29   310239.85   20.276  20.576  20.254     <NA>   304186.33  

[905 rows x 20 columns]

In [34]:
etf=etf.iloc[-4:]

In [40]:
errori = []
errori_nomi = []
#for nome in list(etf.RIC[~etf.Presente]):
for nome in list(etf.RIC):
    try:
        response = historical_pricing.summaries.Definition(
            #   "VWCE.MI",
            nome,
            count=30000,
            interval=historical_pricing.Intervals.DAILY,
            # fields = ["OPEN_PRC","BID","ASK","HIGH_1", "LOW_1", "TRDPRC_1","TRNOVR_UNS"]
            fields = ["OPEN_PRC","HIGH_1", "LOW_1", "TRDPRC_1","TRNOVR_UNS"]
        ).get_data()
        response.data.df.to_csv("dati\\" + nome + ".csv",sep=";")
    except rd.errors.RDError as e:
        try:
            response = historical_pricing.summaries.Definition(
                #   "VWCE.MI",
                nome.replace(".MI",".PA"),
                count=30000,
                interval=historical_pricing.Intervals.DAILY,
                # fields = ["OPEN_PRC","BID","ASK","HIGH_1", "LOW_1", "TRDPRC_1","TRNOVR_UNS"]
                fields = ["OPEN_PRC","HIGH_1", "LOW_1", "TRDPRC_1","TRNOVR_UNS"]
            ).get_data()
            response.data.df.to_csv("dati\\" + nome + ".csv",sep=";")
        except:
            print("Error code :", e.code," Message :", e.message)
            errori.append((e.code,e.message))
            errori_nomi.append(nome)

Session is not opened. Can't send any request


ValueError: Session is not opened. Can't send any request

In [36]:
print(errori_nomi)

[]


In [37]:
pd.Series(errori_nomi).to_csv("etf_errati.csv")

C:\Users\Paolo\AppData\Local\Temp\ipykernel_12132\1295873992.py:1:FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [38]:
rd.close_session()

In [39]:
etf

,Name,Borsa,Presente,ISIN,RIC,Lipper Classification Scheme,Instrument Type,Domicile,Currency,Exchange iNAV
1821,ETC Group Physical Bitcoin,XETRA,True,NaN,BTCE.DE,NaN,ETN,NaN,Euro,NaN
1822,21Shares Bitcoin ETP,XETRA,True,NaN,2BTC.DE,NaN,ETN,NaN,Euro,NaN
1823,21Shares Ethereum Staking ETP,XETRA,True,NaN,ETHA.DE,NaN,ETN,NaN,Euro,NaN
1824,ETC Group Physical Ethereum,XETRA,True,NaN,ZETH.DE,NaN,ETN,NaN,Euro,NaN


# Pezzi che per ora non servono

In [ ]:
response = historical_pricing.events.Definition("VOD.L").get_data()
# Extract in DataFrame format
response.data.df

In [ ]:
# JSON format (display only 200 characters of headers + data to minimise output)
print(json.dumps(response.data.raw['universe'], indent=2),
      'Headers:',
      json.dumps(response.data.raw['headers'], indent=2)[0:200],
      'Data:',
      json.dumps(response.data.raw['data'], indent=2)[0:200])

In [ ]:
response = historical_pricing.events.Definition(
    universe = "AAPL.O", 
    start = datetime.timedelta(-1), 
    end = datetime.timedelta(0),
    adjustments = [
        historical_pricing.Adjustments.EXCHANGE_CORRECTION,
        historical_pricing.Adjustments.MANUAL_CORRECTION
    ]).get_data()
# Extract in DataFrame format
response.data.df

In [ ]:
response = historical_pricing.summaries.Definition(
    universe = "IBM.N", 
    interval = historical_pricing.Intervals.ONE_MINUTE,     # Supported intervals: ONE_MINUTE, FIVE_MINUTES, TEN_MINUTES, THIRTY_MINUTES, ONE_HOUR
    count = 500,
    sessions = [
        historical_pricing.MarketSession.PRE, 
        historical_pricing.MarketSession.NORMAL, 
        historical_pricing.MarketSession.POST
    ]).get_data()
# Extract in DataFrame format
response.data.df

In [ ]:
response = historical_pricing.summaries.Definition("IBM.N").get_data()
# Extract in DataFrame format
response.data.df

In [ ]:
response = historical_pricing.summaries.Definition(
    universe = "VOD.L",
    interval = historical_pricing.Intervals.DAILY,          # Supported intervals: DAILY, WEEKLY, MONTHLY, QUARTERLY, YEARLY.
    count = 10,
    fields = ["BID", "ASK", "OPEN_PRC", "HIGH_1", "LOW_1", "TRDPRC_1", "NUM_MOVES", "TRNOVR_UNS"]
).get_data()
# Extract in DataFrame format
response.data.df

In [ ]:
try:
    response = historical_pricing.events.Definition("BADRIC").get_data()
    print(response.data.df)
except rd.errors.RDError as e:
    print("Error code :", e.code)
    print("Error message:", e.message)

# Tentativo di farlo funzionare senza installare nulla

In [ ]:
# session = rd.session.desktop.Definition(app_key = APP_KEY).get_session()
session = rd.session.platform.Definition(
    app_key = APP_KEY, 
    grant = rd.session.platform.GrantPassword(
        username = RDP_LOGIN, 
        password = RDP_PASSWORD
    )
).get_session()

In [ ]:
# rd.open_session()
# rd.close_session()

In [ ]:
session.open()

In [ ]:
session.set_log_level(logging.DEBUG)
session.open()
# To set logging back to default value
session.set_log_level(logging.WARNING)

In [ ]:
# Callback to handle session state changes
def on_state_cb(state_code, state_msg, session):
    print(f"Session State:{state_code} | {state_msg}")
# Callback to handle session events
def on_event_cb(event_code, event_msg, session):
    print(f"Session event:{event_code} | {event_msg}")

# Set your callbacks for handing the session events
session.on_event(on_event_cb)
session.on_state(on_state_cb)

In [ ]:
session.info

In [ ]:
session.close()